# 가설 2
 - 가설 1차에서 입력 문장을 벡터 / BM25로 검색해봄.
 - 검색된 문장 자체로는 큰 의미 없는 문장이 나오는거 같아, 형태소 단위로 분석해 볼 예정

## 입력 문장 형태소 분리
 - kiwipiepy 사용

In [16]:
!uv pip install kiwipiepy

Resolved 4 packages in 2.18s                                         
Prepared 2 packages in 5.91s                                             
Installed 2 packages in 1ms                                 
 + kiwipiepy==0.22.2
 + kiwipiepy-model==0.22.1


In [24]:
from kiwipiepy import Kiwi

In [34]:
# 키위 태그
from dataclasses import dataclass
from typing import Dict

@dataclass(frozen=True)
class TagInfo:
    desc: str
    group: str
    score: int  # 0~100

TAG_INFO: Dict[str, TagInfo] = {
    # =========================
    # 체언(N): 문장 성분(주어/목적어/보어 등)의 핵심 재료
    # =========================
    "NNG": TagInfo(desc="일반 명사", group="체언(N)", score=90),
    "NNP": TagInfo(desc="고유 명사", group="체언(N)", score=85),
    "NNB": TagInfo(desc="의존 명사", group="체언(N)", score=75),
    "NR":  TagInfo(desc="수사", group="체언(N)", score=70),
    "NP":  TagInfo(desc="대명사", group="체언(N)", score=80),

    # =========================
    # 용언(V): 서술의 핵심(동작/상태/판정)
    # =========================
    "VV":  TagInfo(desc="동사", group="용언(V)", score=90),
    "VA":  TagInfo(desc="형용사", group="용언(V)", score=90),
    "VX":  TagInfo(desc="보조 용언", group="용언(V)", score=65),
    "VCP": TagInfo(desc="긍정 지정사(이다)", group="용언(V)", score=85),
    "VCN": TagInfo(desc="부정 지정사(아니다)", group="용언(V)", score=85),

    # =========================
    # 관형사: 체언 수식(정보 추가) — 핵심 성분보단 부수적
    # =========================
    "MM":  TagInfo(desc="관형사", group="관형사", score=50),

    # =========================
    # 부사(MA): 용언/문장 수식(상황/정도/연결) — 선택적이지만 유용
    # =========================
    "MAG": TagInfo(desc="일반 부사", group="부사(MA)", score=50),
    "MAJ": TagInfo(desc="접속 부사", group="부사(MA)", score=45),

    # =========================
    # 감탄사: 독립 발화 가능 — 문장 성분 구성에는 낮은 기여
    # =========================
    "IC":  TagInfo(desc="감탄사", group="감탄사", score=15),

    # =========================
    # 조사(J): 격/관계 표지(주격, 목적격 등) — 한국어에서 구조 해석에 매우 핵심
    # =========================
    "JKS": TagInfo(desc="주격 조사", group="조사(J)", score=90),
    "JKC": TagInfo(desc="보격 조사", group="조사(J)", score=85),
    "JKG": TagInfo(desc="관형격 조사", group="조사(J)", score=80),
    "JKO": TagInfo(desc="목적격 조사", group="조사(J)", score=90),
    "JKB": TagInfo(desc="부사격 조사", group="조사(J)", score=80),
    "JKV": TagInfo(desc="호격 조사", group="조사(J)", score=45),
    "JKQ": TagInfo(desc="인용격 조사", group="조사(J)", score=75),
    "JX":  TagInfo(desc="보조사", group="조사(J)", score=60),
    "JC":  TagInfo(desc="접속 조사", group="조사(J)", score=70),

    # =========================
    # 어미(E): 문장 종결/연결/시제/높임 등 문법 중심축 — 매우 중요
    # =========================
    "EP":  TagInfo(desc="선어말 어미", group="어미(E)", score=90),
    "EF":  TagInfo(desc="종결 어미", group="어미(E)", score=95),
    "EC":  TagInfo(desc="연결 어미", group="어미(E)", score=90),
    "ETN": TagInfo(desc="명사형 전성 어미", group="어미(E)", score=80),
    "ETM": TagInfo(desc="관형형 전성 어미", group="어미(E)", score=80),

    # =========================
    # 접사/어근: 단어 형성(파생) — 문장 골격보단 어휘 확장에 더 기여
    # =========================
    "XPN": TagInfo(desc="체언 접두사", group="접두사", score=35),
    "XSN": TagInfo(desc="명사 파생 접미사", group="접미사(XS)", score=40),
    "XSV": TagInfo(desc="동사 파생 접미사", group="접미사(XS)", score=40),
    "XSA": TagInfo(desc="형용사 파생 접미사", group="접미사(XS)", score=40),
    "XSM": TagInfo(desc="부사 파생 접미사", group="접미사(XS)", score=35),
    "XR":  TagInfo(desc="어근", group="어근", score=50),

    # =========================
    # 부호/외국어/특수문자(S): 구어 문장 자체엔 필수는 아니나, '문장(텍스트) 표현'에는 유의미
    # =========================
    "SF":  TagInfo(desc="종결 부호(. ! ?)", group="부호, 외국어, 특수문자(S)", score=25),
    "SP":  TagInfo(desc="구분 부호(, / : ;)", group="부호, 외국어, 특수문자(S)", score=15),
    "SS":  TagInfo(desc="인용 부호 및 괄호(' \" ( ) [ ] < > { } ― ‘ ’ “ ” ≪ ≫ 등)", group="부호, 외국어, 특수문자(S)", score=15),
    "SSO": TagInfo(desc="SS 중 여는 부호", group="부호, 외국어, 특수문자(S)", score=10), # SS로 통일
    "SSC": TagInfo(desc="SS 중 닫는 부호", group="부호, 외국어, 특수문자(S)", score=10), # SS로 통일
    "SE":  TagInfo(desc="줄임표(…)", group="부호, 외국어, 특수문자(S)", score=10),
    "SO":  TagInfo(desc="붙임표(- ~)", group="부호, 외국어, 특수문자(S)", score=10),
    "SW":  TagInfo(desc="기타 특수 문자", group="부호, 외국어, 특수문자(S)", score=5),
    "SL":  TagInfo(desc="알파벳(A-Z a-z)", group="부호, 외국어, 특수문자(S)", score=5),
    "SH":  TagInfo(desc="한자", group="부호, 외국어, 특수문자(S)", score=5),
    "SN":  TagInfo(desc="숫자(0-9)", group="부호, 외국어, 특수문자(S)", score=10),
    "SB":  TagInfo(desc="순서 있는 글머리(가. 나. 1. 2. 가) 나) 등)", group="부호, 외국어, 특수문자(S)", score=10), # SN로 통일

    # =========================
    # 분석 불능: 문장 구성 요소로 활용 불가
    # =========================
    "UN":  TagInfo(desc="분석 불능", group="분석 불능", score=0), # NA

    # =========================
    # 웹(W): 문장(언어) 자체라기보다 토큰 유형
    # =========================
    "W_URL":     TagInfo(desc="URL 주소", group="웹(W)", score=5), # 삭제
    "W_EMAIL":   TagInfo(desc="이메일 주소", group="웹(W)", score=5), # 삭제
    "W_HASHTAG": TagInfo(desc="해시태그(#abcd)", group="웹(W)", score=5), # 삭제
    "W_MENTION": TagInfo(desc="멘션(@abcd)", group="웹(W)", score=5), # 삭제
    "W_SERIAL":  TagInfo(desc="일련번호(전화번호, 통장번호, IP주소 등)", group="웹(W)", score=5), # 삭제
    "W_EMOJI":   TagInfo(desc="이모지", group="웹(W)", score=5), # 삭제

    # =========================
    # 기타: 특수 표기(사이시옷/덧받침 등) — 표기/형태론 처리에만 국소적으로 중요
    # =========================
    "Z_CODA": TagInfo(desc="덧붙은 받침", group="기타", score=20), # 삭제
    "Z_SIOT": TagInfo(desc="사이시옷", group="기타", score=20), # 삭제
    "USER0~4": TagInfo(desc="사용자 정의 태그", group="기타", score=0), # 삭제
}

def tag(tag_code: str) -> TagInfo:
    """func('NNG').desc / .group / .score"""
    try:
        return TAG_INFO[tag_code]
    except KeyError as e:
        raise KeyError(f"Unknown tag: {tag_code!r}") from e


In [15]:
# kiwipiepy 태그 → 말뭉치 태그 변환
TAG_CONVERT = {"SSO": "SS", "SSC": "SS", "SB": "SN", "UN": "NA"}
TAG_REMOVE = {"W_URL", "W_EMAIL", "W_HASHTAG", "W_MENTION", "W_SERIAL", "W_EMOJI", 
              "Z_CODA", "Z_SIOT", "USER0", "USER1", "USER2", "USER3", "USER4"}

def preprocess_kiwi(tokens):
    result = []
    for token in tokens:
        tag = token.tag
        form = token.form
        # 삭제 대상 태그는 건너뛰기
        if tag in TAG_REMOVE:
            continue
        # 변환 대상 태그는 변환
        converted_tag = TAG_CONVERT.get(tag, tag)
        result.append((form, converted_tag, token.start, token.len))
    return result



In [16]:
text = "공부가 힘든입니다"

kiwi = Kiwi()
result = kiwi.tokenize(text)

result = preprocess_kiwi(result)
for it in result:
    print(it)

('공부', 'NNG', 0, 2)
('가', 'JKS', 2, 1)
('힘들', 'VA', 4, 2)
('ᆫ', 'ETM', 5, 1)
('이', 'NNB', 6, 1)
('이', 'VCP', 6, 1)
('ᆸ니다', 'EF', 6, 3)


# 말뭉치와 품사태깅 비교

In [5]:
import pandas as pd
# 말뭉치 불러오기
df = pd.read_parquet('말뭉치.parquet.gzip')
# 표본 번호와 문장으로 그루핑
groupped = df.groupby(['표본 번호', '문장'])

In [20]:
arr = []
for (i, sentence), df in groupped:
    print(sentence)
    for it in df.iterrows():
        if it[1]['형태 주석'] != '0':
            arr.append(it[1]['형태 주석'])
    break

그래서 열두 시에 다 끝난면 자도 돼요.


In [21]:
result = kiwi.tokenize("그래서 열두 시에 다 끝난면 자도 돼요.")

result = preprocess_kiwi(result)

kiwi_arr = []

for it in result:
    kiwi_arr.append(it[1])
    

In [22]:
arr

['MAJ', 'MM', 'NNB', 'JKB', 'MAG', 'VV', 'EC', 'VV', 'EC', 'VV', 'EF', 'SF']

In [23]:
kiwi_arr

['MAJ',
 'MM',
 'NNB',
 'JKB',
 'MAG',
 'VV',
 'ETM',
 'NNG',
 'VV',
 'EC',
 'VV',
 'EF',
 'SF']

## TODO
 - 말뭉치의 품사태그와 키위 분석기 품사태그와 약간 다른 부분이 있어서, 말뭉치 문서와 키위 분석기 태그 설명을 보고 맞춰보아야 함.
 - 일단은 skip 하고, 형태주석 / 키위분석기를 N-gram 기반으로 비교하는 테스트 함수 작성

In [32]:
from rapidfuzz.distance import Levenshtein

# 품사태그 → 상위 계열 매핑
TAG_CATEGORY = {
    # 체언 (N)
    'NNG': 'N', 'NNP': 'N', 'NNB': 'N', 'NR': 'N', 'NP': 'N',
    # 용언 (V)
    'VV': 'V', 'VA': 'V', 'VX': 'V', 'VCP': 'V', 'VCN': 'V',
    # 조사 (J)
    'JKS': 'J', 'JKC': 'J', 'JKG': 'J', 'JKO': 'J',
    'JKB': 'J', 'JKV': 'J', 'JKQ': 'J', 'JX': 'J', 'JC': 'J',
    # 어미 (E)
    'EC': 'E', 'EF': 'E', 'EP': 'E', 'ETN': 'E', 'ETM': 'E',
    # 관형사·부사 (M)
    'MM': 'M', 'MAG': 'M', 'MAJ': 'M',
    # 감탄사 (I)
    'IC': 'I',
    # 부호 (S)
    'SF': 'S', 'SP': 'S', 'SS': 'S', 'SE': 'S', 'SO': 'S', 'SW': 'S',
    # 접사·어근 (X)
    'XPN': 'X', 'XSN': 'X', 'XSV': 'X', 'XSA': 'X', 'XR': 'X',
}


def to_categories(arr):
    """품사태그 배열을 상위 계열 배열로 변환"""
    return [TAG_CATEGORY.get(tag, tag) for tag in arr]


def compare_morphs(arr1, arr2, n=2):
    """두 형태주석 배열을 원본/계열 두 수준으로 비교"""
    def calc(a, b):
        dist = Levenshtein.distance(a, b)
        max_len = max(len(a), len(b))
        sim = 1 - (dist / max_len) if max_len > 0 else 1.0

        def get_ngrams(a, n):
            return [tuple(a[i:i+n]) for i in range(len(a)-n+1)]
        ng1, ng2 = set(get_ngrams(a, n)), set(get_ngrams(b, n))
        ng_sim = len(ng1 & ng2) / max(len(ng1), len(ng2)) if ng1 and ng2 else 0.0

        return dist, round(sim, 4), round(ng_sim, 4)

    # 원본 태그 비교 (세밀)
    raw_dist, raw_sim, raw_ng = calc(arr1, arr2)
    # 계열 매핑 후 비교 (거시적)
    cat_dist, cat_sim, cat_ng = calc(to_categories(arr1), to_categories(arr2))

    return {
        "raw_levenshtein_distance": raw_dist,      # 낮을수록 유사 (0 = 완전 일치)
        "raw_levenshtein_similarity": raw_sim,      # 높을수록 유사 (1.0 = 완전 일치)
        "raw_ngram_similarity": raw_ng,             # 높을수록 유사 (1.0 = 완전 일치)
        "cat_levenshtein_distance": cat_dist,       # 낮을수록 유사 (계열 수준)
        "cat_levenshtein_similarity": cat_sim,      # 높을수록 유사 (계열 수준)
        "cat_ngram_similarity": cat_ng,             # 높을수록 유사 (계열 수준)
    }


# 테스트: 말뭉치 형태주석 vs kiwipiepy 분석 결과
result = compare_morphs(arr, kiwi_arr, n=2)
print("말뭉치 형태주석:", arr)
print("말뭉치 계열:    ", to_categories(arr))
print("kiwipiepy 태그: ", kiwi_arr)
print("kiwipiepy 계열: ", to_categories(kiwi_arr))
print()
for k, v in result.items():
    print(f"  {k}: {v}")

말뭉치 형태주석: ['MAJ', 'MM', 'NNB', 'JKB', 'MAG', 'VV', 'EC', 'VV', 'EC', 'VV', 'EF', 'SF']
말뭉치 계열:     ['M', 'M', 'N', 'J', 'M', 'V', 'E', 'V', 'E', 'V', 'E', 'S']
kiwipiepy 태그:  ['MAJ', 'MM', 'NNB', 'JKB', 'MAG', 'VV', 'ETM', 'NNG', 'VV', 'EC', 'VV', 'EF', 'SF']
kiwipiepy 계열:  ['M', 'M', 'N', 'J', 'M', 'V', 'E', 'N', 'V', 'E', 'V', 'E', 'S']

  raw_levenshtein_distance: 2
  raw_levenshtein_similarity: 0.8462
  raw_ngram_similarity: 0.75
  cat_levenshtein_distance: 1
  cat_levenshtein_similarity: 0.9231
  cat_ngram_similarity: 0.8


# 실제 검색 테스트
 - OpenSearch에서 상위 20개의 결과를 모두 가져와, 검색된 말뭉치들의 품사태그와 입력한 텍스트의 품사들을 N-gram 기반으로 비교
 - TAG_INFO에 명시된 정보를 이용하여, 중요도가 높은 품사 태그엔 가점을 크게 주어야 함.
 - 비슷한 그룹끼리 그루핑 하여 체크도 해야 함.

## 구현 계획

### 목표
입력 문장의 오류 유형(오타/문법/혼합)을 판별하고, 말뭉치 내 유사 오류 사례를 검색

### 핵심 로직
1. **입력 전처리**: kiwipiepy로 형태소 분석 → 품사 태그 배열 생성
2. **OpenSearch 검색**: BM25 + 벡터 검색 각각 상위 20개 조회
3. **형태소 유사도 계산**: TAG_INFO 가중치 반영한 N-gram 비교
4. **오류 유형 추정**: BM25/벡터 스코어 + 형태소 유사도 종합 분석

### 흐름도

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                           입력 문장                                          │
│                    "저는 어제 뱡완에 갔어요."                                  │
└───────────────────────────────┬─────────────────────────────────────────────┘
                                │
                                ▼
┌─────────────────────────────────────────────────────────────────────────────┐
│                      [1] 입력 전처리 (kiwipiepy)                             │
├─────────────────────────────────────────────────────────────────────────────┤
│  형태소 분석 → 품사 태그 배열: ['NP', 'JX', 'MAG', 'NNG', 'JKB', ...]       │
│  임베딩 벡터 생성: model.encode(입력문장) → [1024차원 벡터]                   │
└───────────────────────────────┬─────────────────────────────────────────────┘
                                │
        ┌───────────────────────┴───────────────────────┐
        ▼                                               ▼
┌───────────────────────────┐               ┌───────────────────────────┐
│  [2-A] BM25 검색 (Top 20)  │               │  [2-B] 벡터 검색 (Top 20)  │
│  키워드 매칭 기반          │               │  의미 유사도 기반          │
└─────────────┬─────────────┘               └─────────────┬─────────────┘
              └─────────────────┬─────────────────────────┘
                                ▼
┌─────────────────────────────────────────────────────────────────────────────┐
│                    [3] 형태소 유사도 계산 (각 결과에 대해)                     │
├─────────────────────────────────────────────────────────────────────────────┤
│  (A) 레벤슈타인 거리: raw(원본 태그) + cat(상위 계열)                        │
│  (B) 가중치 적용 N-gram: TAG_INFO.score로 중요 품사에 가점                   │
└───────────────────────────────┬─────────────────────────────────────────────┘
                                ▼
┌─────────────────────────────────────────────────────────────────────────────┐
│                         [4] 오류 유형 추정                                   │
├─────────────────────────────────────────────────────────────────────────────┤
│  BM25 높음 & 벡터 낮음  →  오타 오류 가능성 ↑                               │
│  BM25 낮음 & 벡터 높음  →  문법 오류 가능성 ↑                               │
│  둘 다 높음 & 형태소 유사  →  동일/매우 유사 문장                            │
└───────────────────────────────┬─────────────────────────────────────────────┘
                                ▼
┌─────────────────────────────────────────────────────────────────────────────┐
│  [5] 종합 스코어 계산 & 결과 출력                                            │
│  final_score = α×norm_bm25 + β×norm_vector + γ×morph_similarity            │
└─────────────────────────────────────────────────────────────────────────────┘
```

In [28]:
from sentence_transformers import SentenceTransformer

# embedding 모델 불러오기
model = SentenceTransformer(
        "./model/KURE-v1",
        local_files_only=True,
)

/home/puju/py_playground/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading weights: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 391/391 [00:01<00:00, 294.96it/s, Materializing param=pooler.dense.weight]


In [25]:
# OpenSearch 클라이언트 연결
from opensearchpy import OpenSearch

client = OpenSearch(
    hosts=[{'host': '172.30.1.81', 'port': 9200}],
    http_auth=None,
    use_ssl=False,
    verify_certs=False
)

INDEX_NAME = 'korean_test'

In [29]:
# [1] 입력 전처리 함수
from dataclasses import dataclass
from typing import List, Tuple

@dataclass
class PreprocessedInput:
    """전처리된 입력 데이터"""
    text: str
    tokens: List[Tuple[str, str, int, int]]  # (form, tag, start, len)
    tags: List[str]  # 품사 태그 배열
    embedding: List[float]  # 1024차원 벡터

def preprocess_input(text: str) -> PreprocessedInput:
    """입력 문장을 형태소 분석하고 임베딩 벡터 생성"""
    # 형태소 분석
    result = kiwi.tokenize(text)
    tokens = preprocess_kiwi(result)
    tags = [t[1] for t in tokens]
    
    # 임베딩 생성
    embedding = model.encode(text).tolist()
    
    return PreprocessedInput(
        text=text,
        tokens=tokens,
        tags=tags,
        embedding=embedding
    )

# 테스트
test_input = preprocess_input("저는 어제 뱡완에 갔어요.")
print(f"입력 문장: {test_input.text}")
print(f"토큰: {test_input.tokens}")
print(f"품사 태그: {test_input.tags}")
print(f"임베딩 차원: {len(test_input.embedding)}")

입력 문장: 저는 어제 뱡완에 갔어요.
토큰: [('저', 'NP', 0, 1), ('는', 'JX', 1, 1), ('어제', 'MAG', 3, 2), ('뱡완', 'NNP', 6, 2), ('에', 'JKB', 8, 1), ('가', 'VV', 10, 1), ('었', 'EP', 10, 1), ('어요', 'EF', 11, 2), ('.', 'SF', 13, 1)]
품사 태그: ['NP', 'JX', 'MAG', 'NNP', 'JKB', 'VV', 'EP', 'EF', 'SF']
임베딩 차원: 1024


In [30]:
# [2] OpenSearch 검색 함수
from typing import Dict, Any

@dataclass
class SearchResult:
    """검색 결과 단일 항목"""
    doc_id: str
    text: str
    morphs: List[str]
    bm25_score: float = 0.0
    vector_score: float = 0.0

def search_bm25(query_text: str, k: int = 20) -> List[SearchResult]:
    """BM25 키워드 검색"""
    query = {
        "size": k,
        "query": {
            "match": {
                "original_text": query_text
            }
        }
    }
    
    results = client.search(index=INDEX_NAME, body=query)
    
    return [
        SearchResult(
            doc_id=hit['_id'],
            text=hit['_source']['original_text'],
            morphs=hit['_source'].get('morphs', []),
            bm25_score=hit['_score']
        )
        for hit in results['hits']['hits']
    ]

def search_vector(embedding: List[float], k: int = 20) -> List[SearchResult]:
    """벡터 kNN 검색"""
    query = {
        "size": k,
        "query": {
            "knn": {
                "embedding": {
                    "vector": embedding,
                    "k": k
                }
            }
        }
    }
    
    results = client.search(index=INDEX_NAME, body=query)
    
    return [
        SearchResult(
            doc_id=hit['_id'],
            text=hit['_source']['original_text'],
            morphs=hit['_source'].get('morphs', []),
            vector_score=hit['_score']
        )
        for hit in results['hits']['hits']
    ]

def search_combined(query_text: str, embedding: List[float], k: int = 20) -> Dict[str, SearchResult]:
    """BM25 + 벡터 검색 결과 병합"""
    bm25_results = search_bm25(query_text, k)
    vector_results = search_vector(embedding, k)
    
    # doc_id 기준으로 병합
    combined: Dict[str, SearchResult] = {}
    
    for r in bm25_results:
        combined[r.doc_id] = r
    
    for r in vector_results:
        if r.doc_id in combined:
            combined[r.doc_id].vector_score = r.vector_score
        else:
            combined[r.doc_id] = r
    
    return combined

# 테스트
print("[BM25 검색 결과 Top 5]")
bm25_results = search_bm25(test_input.text, k=5)
for r in bm25_results:
    print(f"  {r.bm25_score:.4f} | {r.text}")

print("\n[벡터 검색 결과 Top 5]")
vector_results = search_vector(test_input.embedding, k=5)
for r in vector_results:
    print(f"  {r.vector_score:.4f} | {r.text}")

[BM25 검색 결과 Top 5]
  20.1484 | 휴일이었지요 어제 학교에 갔어요
  16.6537 | 저는 네팔에 보커라 갔어요.
  16.1505 | 좀 비군해서 저는 집에 갔어요.
  16.1505 | 저는 친구하고 같이 인사동에 갔어요.
  15.3742 | 친구들을 제 집에 갔어요.

[벡터 검색 결과 Top 5]
  0.8460 | 페이징 상하에 지난내 갔어요
  0.8421 | 주말에 제주도 갔어요.
  0.8358 | 항쿡에 가 봤어요
  0.8355 | 해운대에 갔어요
  0.8351 | 첫날에는 저는 유니버첼스튜디오에 갔습니다.


In [35]:
# [3] 형태소 유사도 계산 함수 (가중치 적용)

def get_tag_weight(tag: str) -> float:
    """TAG_INFO에서 품사 태그의 가중치(0~1) 반환"""
    if tag in TAG_INFO:
        return TAG_INFO[tag].score / 100.0
    return 0.5  # 알 수 없는 태그는 중간값

def get_ngrams(arr: List[str], n: int = 2) -> List[Tuple[str, ...]]:
    """N-gram 생성"""
    return [tuple(arr[i:i+n]) for i in range(len(arr)-n+1)]

def weighted_ngram_similarity(tags1: List[str], tags2: List[str], n: int = 2) -> float:
    """가중치 적용 N-gram 유사도 계산"""
    if len(tags1) < n or len(tags2) < n:
        return 0.0
    
    ngrams1 = get_ngrams(tags1, n)
    ngrams2 = get_ngrams(tags2, n)
    
    # 각 N-gram의 가중치 계산 (N-gram 내 태그들의 평균 가중치)
    def ngram_weight(ngram: Tuple[str, ...]) -> float:
        return sum(get_tag_weight(t) for t in ngram) / len(ngram)
    
    # 가중치 적용 교집합 계산
    set1 = set(ngrams1)
    set2 = set(ngrams2)
    common = set1 & set2
    
    if not common:
        return 0.0
    
    # 공통 N-gram의 가중치 합
    common_weight = sum(ngram_weight(ng) for ng in common)
    
    # 전체 N-gram의 가중치 합 (더 큰 쪽 기준)
    total_weight = max(
        sum(ngram_weight(ng) for ng in set1),
        sum(ngram_weight(ng) for ng in set2)
    )
    
    return common_weight / total_weight if total_weight > 0 else 0.0

def weighted_levenshtein_similarity(tags1: List[str], tags2: List[str]) -> float:
    """가중치 적용 레벤슈타인 유사도 계산"""
    if not tags1 or not tags2:
        return 0.0
    
    # 기본 레벤슈타인 거리
    dist = Levenshtein.distance(tags1, tags2)
    max_len = max(len(tags1), len(tags2))
    base_sim = 1 - (dist / max_len)
    
    # 중요 품사 일치 보너스 계산
    important_tags = {'JKS', 'JKO', 'EF', 'EC', 'VV', 'VA', 'NNG', 'NNP'}
    
    imp1 = [t for t in tags1 if t in important_tags]
    imp2 = [t for t in tags2 if t in important_tags]
    
    if imp1 and imp2:
        imp_common = len(set(imp1) & set(imp2))
        imp_total = max(len(set(imp1)), len(set(imp2)))
        imp_bonus = (imp_common / imp_total) * 0.1  # 최대 10% 보너스
    else:
        imp_bonus = 0.0
    
    return min(base_sim + imp_bonus, 1.0)

@dataclass
class MorphSimilarity:
    """형태소 유사도 결과"""
    raw_levenshtein: float  # 원본 태그 레벤슈타인 유사도
    cat_levenshtein: float  # 계열 태그 레벤슈타인 유사도
    weighted_ngram: float   # 가중치 적용 N-gram 유사도
    combined: float         # 종합 유사도

def calculate_morph_similarity(input_tags: List[str], result_tags: List[str]) -> MorphSimilarity:
    """형태소 유사도 종합 계산"""
    if not result_tags:
        return MorphSimilarity(0.0, 0.0, 0.0, 0.0)
    
    # 원본 태그 레벤슈타인 유사도
    raw_lev = weighted_levenshtein_similarity(input_tags, result_tags)
    
    # 계열 변환 후 레벤슈타인 유사도
    cat1 = to_categories(input_tags)
    cat2 = to_categories(result_tags)
    cat_lev = weighted_levenshtein_similarity(cat1, cat2)
    
    # 가중치 적용 N-gram 유사도 (2-gram, 3-gram 평균)
    ng2 = weighted_ngram_similarity(input_tags, result_tags, n=2)
    ng3 = weighted_ngram_similarity(input_tags, result_tags, n=3)
    weighted_ng = (ng2 * 0.6 + ng3 * 0.4)  # 2-gram에 더 큰 가중치
    
    # 종합 유사도 (가중 평균)
    combined = (raw_lev * 0.3 + cat_lev * 0.3 + weighted_ng * 0.4)
    
    return MorphSimilarity(
        raw_levenshtein=round(raw_lev, 4),
        cat_levenshtein=round(cat_lev, 4),
        weighted_ngram=round(weighted_ng, 4),
        combined=round(combined, 4)
    )

# 테스트
print("입력 품사 태그:", test_input.tags)
print()
for r in bm25_results[:3]:
    sim = calculate_morph_similarity(test_input.tags, r.morphs)
    print(f"[{r.text}]")
    print(f"  결과 morphs: {r.morphs}")
    print(f"  유사도: raw_lev={sim.raw_levenshtein}, cat_lev={sim.cat_levenshtein}, "
          f"weighted_ng={sim.weighted_ngram}, combined={sim.combined}")
    print()

입력 품사 태그: ['NP', 'JX', 'MAG', 'NNP', 'JKB', 'VV', 'EP', 'EF', 'SF']

[휴일이었지요 어제 학교에 갔어요]
  결과 morphs: ['NNG', 'VCP', 'EP', 'EF', 'MAG', 'NNG', 'JKB', 'VV', 'EP', 'EF']
  유사도: raw_lev=0.4667, cat_lev=0.6, weighted_ng=0.3455, combined=0.4582

[저는 네팔에 보커라 갔어요.]
  결과 morphs: ['NP', 'JX', 'NNP', 'JKB', 'NNP', 'VV', 'EP', 'EF', 'SF']
  유사도: raw_lev=0.8778, cat_lev=0.7778, weighted_ng=0.4855, combined=0.6909

[좀 비군해서 저는 집에 갔어요.]
  결과 morphs: ['MAG', 'NNG', 'XSA', 'EC', 'NP', 'JX', 'NNG', 'JKB', 'VV', 'EP', 'EF', 'SF']
  유사도: raw_lev=0.55, cat_lev=0.5833, weighted_ng=0.4124, combined=0.5049



In [36]:
# [4] 오류 유형 추정 함수
from enum import Enum

class ErrorType(Enum):
    TYPO = "오타 오류"           # BM25 높고 벡터 낮음
    GRAMMAR = "문법 오류"        # BM25 낮고 벡터 높음
    MIXED = "혼합 오류"          # 둘 다 중간
    SIMILAR = "유사 문장"        # 둘 다 높음
    UNRELATED = "관련성 낮음"    # 둘 다 낮음

def normalize_bm25_score(score: float, max_score: float = 15.0) -> float:
    """BM25 스코어를 0~1로 정규화"""
    return min(score / max_score, 1.0)

def estimate_error_type(
    norm_bm25: float, 
    norm_vector: float, 
    morph_sim: float,
    threshold: float = 0.15
) -> Tuple[ErrorType, str]:
    """
    오류 유형 추정
    
    Args:
        norm_bm25: 정규화된 BM25 스코어 (0~1)
        norm_vector: 벡터 유사도 스코어 (0~1)
        morph_sim: 형태소 유사도 (0~1)
        threshold: 판별 임계값
    
    Returns:
        (오류유형, 판별근거)
    """
    diff = norm_bm25 - norm_vector
    
    # 둘 다 매우 높으면 유사 문장
    if norm_bm25 > 0.7 and norm_vector > 0.8:
        return ErrorType.SIMILAR, f"BM25({norm_bm25:.2f})와 벡터({norm_vector:.2f}) 모두 높음"
    
    # 둘 다 낮으면 관련성 낮음
    if norm_bm25 < 0.3 and norm_vector < 0.6:
        return ErrorType.UNRELATED, f"BM25({norm_bm25:.2f})와 벡터({norm_vector:.2f}) 모두 낮음"
    
    # BM25 >> 벡터: 오타 오류 가능성
    if diff > threshold:
        return ErrorType.TYPO, f"BM25({norm_bm25:.2f}) > 벡터({norm_vector:.2f}), 차이={diff:.2f}"
    
    # 벡터 >> BM25: 문법 오류 가능성
    if diff < -threshold:
        return ErrorType.GRAMMAR, f"벡터({norm_vector:.2f}) > BM25({norm_bm25:.2f}), 차이={-diff:.2f}"
    
    # 그 외 혼합
    return ErrorType.MIXED, f"BM25({norm_bm25:.2f}) ≈ 벡터({norm_vector:.2f}), 형태소유사도={morph_sim:.2f}"

In [37]:
# [5] 종합 분석 함수

@dataclass
class AnalysisResult:
    """분석 결과 단일 항목"""
    doc_id: str
    text: str
    morphs: List[str]
    bm25_score: float
    vector_score: float
    norm_bm25: float
    norm_vector: float
    morph_similarity: MorphSimilarity
    error_type: ErrorType
    error_reason: str
    final_score: float

def analyze_search_results(
    input_data: PreprocessedInput,
    k: int = 20,
    alpha: float = 0.3,  # BM25 가중치
    beta: float = 0.3,   # 벡터 가중치
    gamma: float = 0.4   # 형태소 유사도 가중치
) -> List[AnalysisResult]:
    """
    검색 및 종합 분석 수행
    
    Args:
        input_data: 전처리된 입력 데이터
        k: 검색 결과 개수
        alpha, beta, gamma: 최종 스코어 계산 가중치
    
    Returns:
        분석 결과 리스트 (final_score 내림차순 정렬)
    """
    # 검색 수행
    combined = search_combined(input_data.text, input_data.embedding, k)
    
    # BM25 스코어 최대값 찾기 (정규화용)
    max_bm25 = max((r.bm25_score for r in combined.values()), default=1.0)
    max_bm25 = max(max_bm25, 1.0)  # 0 방지
    
    results = []
    
    for doc_id, sr in combined.items():
        # 스코어 정규화
        norm_bm25 = sr.bm25_score / max_bm25
        norm_vector = sr.vector_score  # 이미 0~1 범위
        
        # 형태소 유사도 계산
        morph_sim = calculate_morph_similarity(input_data.tags, sr.morphs)
        
        # 오류 유형 추정
        error_type, error_reason = estimate_error_type(
            norm_bm25, norm_vector, morph_sim.combined
        )
        
        # 최종 스코어 계산
        final_score = (
            alpha * norm_bm25 +
            beta * norm_vector +
            gamma * morph_sim.combined
        )
        
        results.append(AnalysisResult(
            doc_id=doc_id,
            text=sr.text,
            morphs=sr.morphs,
            bm25_score=sr.bm25_score,
            vector_score=sr.vector_score,
            norm_bm25=round(norm_bm25, 4),
            norm_vector=round(norm_vector, 4),
            morph_similarity=morph_sim,
            error_type=error_type,
            error_reason=error_reason,
            final_score=round(final_score, 4)
        ))
    
    # final_score 내림차순 정렬
    results.sort(key=lambda x: x.final_score, reverse=True)
    
    return results

In [38]:
# [6] 결과 출력 함수

def print_analysis_results(
    input_data: PreprocessedInput,
    results: List[AnalysisResult],
    top_n: int = 10
):
    """분석 결과 출력"""
    print("=" * 80)
    print(f"입력 문장: {input_data.text}")
    print(f"입력 품사: {input_data.tags}")
    print("=" * 80)
    
    # 오류 유형별 집계
    type_counts = {}
    for r in results:
        t = r.error_type.value
        type_counts[t] = type_counts.get(t, 0) + 1
    
    print("\n[오류 유형 분포]")
    for t, cnt in sorted(type_counts.items(), key=lambda x: -x[1]):
        print(f"  {t}: {cnt}건")
    
    # 상위 결과 출력
    print(f"\n[상위 {top_n}개 유사 오류 사례]")
    print("-" * 80)
    print(f"{'순위':^4} | {'종합':^6} | {'BM25':^6} | {'벡터':^6} | {'형태소':^6} | {'유형':^10} | 문장")
    print("-" * 80)
    
    for i, r in enumerate(results[:top_n], 1):
        print(f"{i:^4} | {r.final_score:^6.3f} | {r.norm_bm25:^6.3f} | "
              f"{r.norm_vector:^6.3f} | {r.morph_similarity.combined:^6.3f} | "
              f"{r.error_type.value:^10} | {r.text[:40]}")
    
    print("-" * 80)
    
    # 상위 3개 상세 정보
    print("\n[상위 3개 상세 분석]")
    for i, r in enumerate(results[:3], 1):
        print(f"\n--- {i}위 ---")
        print(f"문장: {r.text}")
        print(f"morphs: {r.morphs}")
        print(f"BM25: {r.bm25_score:.4f} (정규화: {r.norm_bm25:.4f})")
        print(f"벡터: {r.vector_score:.4f} (정규화: {r.norm_vector:.4f})")
        print(f"형태소 유사도: raw_lev={r.morph_similarity.raw_levenshtein}, "
              f"cat_lev={r.morph_similarity.cat_levenshtein}, "
              f"weighted_ng={r.morph_similarity.weighted_ngram}")
        print(f"오류 유형: {r.error_type.value}")
        print(f"판별 근거: {r.error_reason}")

In [39]:
# [7] 통합 검색 함수 (최종 API)

def search_and_analyze(query: str, k: int = 20, top_n: int = 10) -> List[AnalysisResult]:
    """
    입력 문장에 대한 오류 분석 및 유사 사례 검색
    
    Args:
        query: 분석할 문장
        k: 검색 결과 개수
        top_n: 출력할 상위 결과 개수
    
    Returns:
        분석 결과 리스트
    """
    # 1. 입력 전처리
    input_data = preprocess_input(query)
    
    # 2. 검색 및 분석
    results = analyze_search_results(input_data, k=k)
    
    # 3. 결과 출력
    print_analysis_results(input_data, results, top_n=top_n)
    
    return results

## 테스트 실행

In [40]:
# 테스트 1: 오타 오류 문장 (병원 → 뱡완)
results1 = search_and_analyze("저는 어제 뱡완에 갔어요.")

입력 문장: 저는 어제 뱡완에 갔어요.
입력 품사: ['NP', 'JX', 'MAG', 'NNP', 'JKB', 'VV', 'EP', 'EF', 'SF']

[오류 유형 분포]
  문법 오류: 18건
  오타 오류: 16건
  유사 문장: 2건

[상위 10개 유사 오류 사례]
--------------------------------------------------------------------------------
 순위  |   종합   |  BM25  |   벡터   |  형태소   |     유형     | 문장
--------------------------------------------------------------------------------
 1   | 0.712  | 0.743  | 0.828  | 0.603  |   유사 문장    | 저는 홍대 있는 백화점에 갔습니다.
 2   | 0.712  | 0.665  | 0.835  | 0.656  |   문법 오류    | 첫날에는 저는 유니버첼스튜디오에 갔습니다.
 3   | 0.695  | 0.704  | 0.836  | 0.582  |   유사 문장    | 해운대에 갔어요
 4   | 0.683  | 0.677  | 0.842  | 0.568  |   문법 오류    | 주말에 제주도 갔어요.
 5   | 0.551  | 0.802  | 0.000  | 0.776  |   오타 오류    | 저는 친구하고 같이 인사동에 갔어요.
 6   | 0.524  | 0.827  | 0.000  | 0.691  |   오타 오류    | 저는 네팔에 보커라 갔어요.
 7   | 0.483  | 1.000  | 0.000  | 0.458  |   오타 오류    | 휴일이었지요 어제 학교에 갔어요
 8   | 0.482  | 0.000  | 0.832  | 0.582  |   문법 오류    | 홍대에 갔다
 9   | 0.471  | 0.000  | 0.830  | 0.554  |   문법

In [41]:
# 테스트 2: 문법 오류 문장 (조사 오류)
results2 = search_and_analyze("저는 친구를 같이 영화를 봤어요.")

입력 문장: 저는 친구를 같이 영화를 봤어요.
입력 품사: ['NP', 'JX', 'NNG', 'JKO', 'MAG', 'NNG', 'JKO', 'VV', 'EP', 'EF', 'SF']

[오류 유형 분포]
  문법 오류: 18건
  오타 오류: 15건
  유사 문장: 2건

[상위 10개 유사 오류 사례]
--------------------------------------------------------------------------------
 순위  |   종합   |  BM25  |   벡터   |  형태소   |     유형     | 문장
--------------------------------------------------------------------------------
 1   | 0.832  | 1.000  | 0.878  | 0.671  |   유사 문장    | 혼자 영화를 봤어요.
 2   | 0.685  | 0.570  | 0.922  | 0.593  |   문법 오류    | 친구는 영화를 보고 싶었습니다.
 3   | 0.665  | 0.807  | 0.908  | 0.377  |   유사 문장    | 친구들 같이 봤다
 4   | 0.654  | 0.660  | 0.913  | 0.454  |   문법 오류    | 친구와 같이 영화관에서 영화를 볼 거예요.
 5   | 0.642  | 0.613  | 0.926  | 0.450  |   문법 오류    | 친구하고 같이 영화를 보러 가고 싶습니다.
 6   | 0.526  | 0.000  | 0.876  | 0.657  |   문법 오류    | 친구하고 쇼핑을 했어요.
 7   | 0.523  | 0.000  | 0.868  | 0.657  |   문법 오류    | 친구하고 이야기를 했습니다.
 8   | 0.503  | 0.000  | 0.876  | 0.600  |   문법 오류    | 친구하고 같이 집에 갔어요.
 9   | 0.500  | 0.000  

In [42]:
# 테스트 3: 올바른 문장
results3 = search_and_analyze("저는 어제 병원에 갔어요.")

입력 문장: 저는 어제 병원에 갔어요.
입력 품사: ['NP', 'JX', 'MAG', 'NNG', 'JKB', 'VV', 'EP', 'EF', 'SF']

[오류 유형 분포]
  문법 오류: 16건
  오타 오류: 16건
  유사 문장: 4건

[상위 10개 유사 오류 사례]
--------------------------------------------------------------------------------
 순위  |   종합   |  BM25  |   벡터   |  형태소   |     유형     | 문장
--------------------------------------------------------------------------------
 1   | 0.783  | 0.986  | 0.836  | 0.592  |   유사 문장    | 휴일이었지요 어제 학교에 갔어요
 2   | 0.748  | 0.758  | 0.811  | 0.693  |   유사 문장    | 집에 갔어요.
 3   | 0.686  | 0.806  | 0.825  | 0.491  |   유사 문장    | 친구도 부무님도 빨리 병원에 가라고 해서 병원에 갔다.
 4   | 0.623  | 0.818  | 0.844  | 0.310  |   유사 문장    | 저는 작년 여름에 배탈이 심한 데다가 설사도 해서 병원에 갔다.
 5   | 0.519  | 0.000  | 0.822  | 0.681  |   문법 오류    | 호텔에서 한 밤에 지냈었어
 6   | 0.517  | 0.800  | 0.000  | 0.693  |   오타 오류    | 저는 친구하고 같이 인사동에 갔어요.
 7   | 0.514  | 0.000  | 0.851  | 0.648  |   문법 오류    | 지난 주말에 감기에 걸렸습니다.
 8   | 0.506  | 0.000  | 0.823  | 0.648  |   문법 오류    | 지난 주말에 술집에 갔어요.
 9   | 0.503

In [43]:
# 테스트 4: 어미 오류 문장
results4 = search_and_analyze("공부가 힘든입니다.")

입력 문장: 공부가 힘든입니다.
입력 품사: ['NNG', 'JKS', 'VA', 'ETM', 'NNB', 'VCP', 'EF', 'SF']

[오류 유형 분포]
  문법 오류: 18건
  오타 오류: 18건
  유사 문장: 2건

[상위 10개 유사 오류 사례]
--------------------------------------------------------------------------------
 순위  |   종합   |  BM25  |   벡터   |  형태소   |     유형     | 문장
--------------------------------------------------------------------------------
 1   | 0.730  | 0.920  | 0.882  | 0.472  |   유사 문장    | 힘든 것도 있어요.
 2   | 0.717  | 0.958  | 0.897  | 0.402  |   유사 문장    | 힘든 것 같아서
 3   | 0.570  | 0.000  | 0.898  | 0.752  |   문법 오류    | 학생 생활이 좀 어려운 편이다.
 4   | 0.531  | 0.958  | 0.000  | 0.610  |   오타 오류    | 힘든 거예요
 5   | 0.438  | 0.841  | 0.000  | 0.464  |   오타 오류    | 힘든 것이 힘들기 때문이에요 곳 가따요
 6   | 0.422  | 1.000  | 0.000  | 0.306  |   오타 오류    | 힘든 한국어?
 7   | 0.411  | 0.793  | 0.000  | 0.433  |   오타 오류    | 힘든 시간이 췹게 지난 수 있습니다
 8   | 0.406  | 0.741  | 0.000  | 0.460  |   오타 오류    | 한국 생활이 힘든다니까 걱정 많이 했어요.
 9   | 0.406  | 0.000  | 0.891  | 0.348  |   문법 오류    | 학생 생활이 좀

In [44]:
# 내 테스트
results4 = search_and_analyze("내 월꺼 아네 있는데요")

입력 문장: 내 월꺼 아네 있는데요
입력 품사: ['NP', 'JKG', 'NNG', 'VV', 'EC', 'VX', 'EC', 'JX']

[오류 유형 분포]
  오타 오류: 19건
  문법 오류: 19건
  유사 문장: 1건

[상위 10개 유사 오류 사례]
--------------------------------------------------------------------------------
 순위  |   종합   |  BM25  |   벡터   |  형태소   |     유형     | 문장
--------------------------------------------------------------------------------
 1   | 0.614  | 0.820  | 0.817  | 0.308  |   유사 문장    | 한 통장은 내 용돈으로 만든다.
 2   | 0.520  | 1.000  | 0.000  | 0.551  |   오타 오류    | 항상 내 도전이 기다리고 있다
 3   | 0.470  | 0.881  | 0.000  | 0.514  |   오타 오류    | 항상 내 이야기를 자주 들어준다.
 4   | 0.463  | 0.000  | 0.793  | 0.564  |   문법 오류    | 제 집이 좀 보고 싶어요.
 5   | 0.412  | 0.000  | 0.825  | 0.411  |   문법 오류    | 제 생일이 있습니다.
 6   | 0.412  | 0.000  | 0.825  | 0.411  |   문법 오류    | 제 생일이 있습니다.
 7   | 0.402  | 0.000  | 0.799  | 0.406  |   문법 오류    | 제 이름은 <name>입니다.
 8   | 0.402  | 0.000  | 0.795  | 0.408  |   문법 오류    | 저는 돈을 많지 않아요.
 9   | 0.402  | 0.000  | 0.795  | 0.408  |   문법 오류    | 저는 돈

In [50]:
# 내 테스트
results4 = search_and_analyze("제 집에 가고 싶어요.")

입력 문장: 제 집에 가고 싶어요.
입력 품사: ['NP', 'JKG', 'NNG', 'JKB', 'VV', 'EC', 'VX', 'EF', 'SF']

[오류 유형 분포]
  문법 오류: 19건
  오타 오류: 14건
  유사 문장: 1건

[상위 10개 유사 오류 사례]
--------------------------------------------------------------------------------
 순위  |   종합   |  BM25  |   벡터   |  형태소   |     유형     | 문장
--------------------------------------------------------------------------------
 1   | 0.885  | 1.000  | 0.918  | 0.775  |   유사 문장    | 제 집이 좀 보고 싶어요.
 2   | 0.727  | 0.644  | 0.858  | 0.691  |   문법 오류    | 한국에 가고 싶어요.
 3   | 0.671  | 0.643  | 0.856  | 0.552  |   문법 오류    | 친구들을 제 집에 갔어요.
 4   | 0.668  | 0.681  | 0.855  | 0.517  |   문법 오류    | 집에서 쉬기는커녕 집에 가고 싶지 않다.
 5   | 0.651  | 0.599  | 0.905  | 0.500  |   문법 오류    | 집에 갔어요.
 6   | 0.640  | 0.607  | 0.860  | 0.500  |   문법 오류    | 친구하고 같이 집에 갔어요.
 7   | 0.575  | 0.000  | 0.864  | 0.788  |   문법 오류    | 지금 옛날에 돌아가고 싶어요.
 8   | 0.556  | 0.000  | 0.864  | 0.742  |   문법 오류    | 저는 가족은 보고 싶어요.
 9   | 0.549  | 0.000  | 0.859  | 0.730  |   문법 오류    | 

# 테스트 결과
 - 입력 문장과 오류 문장을 단순 비교하는 방식엔 확실히 한계가 있는듯 함.
 - "조금 나아졌다" 뿐이지, 여전히 입력 문장의 에러와 유사한 에러케이스를 찾는데는 힘들어 보임.

# 개선 방안
 - 말뭉치 샘플 내 존재하는 에러 케이스와 교정 형태소를 활용하면 좋을거 같다는 생각이 듬.
 - N-gram 검색을 에러 케이스까지 포함시킨다면?
 - 예를 들면 ...